# Post-Retrieval Optimization:

## Re-ranking with LLM

In [ ]:
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# Assume documents retrieved from FAISS
retrieved_docs: list[Document] = vectorstore.similarity_search(query, k=10)

# Re-ranking function using LLM
def rerank_with_llm(query: str, docs: list[Document], llm) -> list[Document]:
    ranked_docs = []
    for doc in docs:
        prompt = f"Query: {query}\n\nDocument: {doc.page_content}\n\nHow relevant is this document to the query (score 0-10)?"
        score = float(llm(prompt))
        ranked_docs.append((doc, score))
    ranked_docs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in ranked_docs]

llm = OpenAI(temperature=0)
reranked_docs = rerank_with_llm(query, retrieved_docs, llm)


## Contextual Compression (LLM-based summarization)

In [ ]:
# Compress/summarize documents using LLM to fit within token limits
def compress_docs_with_llm(docs: list[Document], llm) -> str:
    summaries = []
    for doc in docs:
        prompt = f"Summarize the key points from the following:\n\n{doc.page_content}"
        summary = llm(prompt)
        summaries.append(summary.strip())
    return "\n\n".join(summaries)

compressed_context = compress_docs_with_llm(reranked_docs[:5], llm)


## RAG-Fusion (Query variants + Reciprocal Rank Fusion)

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Generate query variants
query_variants = [
    f"{query}",
    f"More specific: {query}",
    f"Rephrase: What are details about {query}?",
]

# Retrieve top results for each variant
all_results = []
for q in query_variants:
    docs = vectorstore.similarity_search(q, k=10)
    all_results.append(docs)

# Reciprocal Rank Fusion
from collections import defaultdict

def rrf(docs_lists, k=60, alpha=60):
    scores = defaultdict(float)
    for docs in docs_lists:
        for rank, doc in enumerate(docs):
            scores[doc.page_content] += 1 / (alpha + rank)
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [Document(page_content=doc) for doc, _ in ranked[:k]]

fused_docs = rrf(all_results)

# Optional: re-rank fused docs again
final_docs = rerank_with_llm(query, fused_docs, llm)
